This widget dashboard allows you to plan for a spontaneous weekend trip. First, input the IATA airline code for the airline you want to fly with (Common Examples Include: AA- American Airlines, SW- Southwestern, F9- Frontier Airlines, DL- Delta Airlines, B6- JetBlue, LH- Lufthansa). Next, select the IATA of the airport you want to fly out of, we included the top 50 airports in the US for simplicity. Finally, select the number of one-way flight options that should appear in your output and that you want to see displayed in the graph. This is the link to the API: https://app.goflightlabs.com/dashboard. Sign up, make an account and copy the key in the blank variable name (access_key). You can then switch any of the inputs and after clicking the Show Flights Button again, your updated choices should show up.

We made use of a number of additional libraries in our project. The first and most important library to install is the networkx library. This library provides a variety of tools surrounding the graph data structure. I am sure you are familiar with how this data structure works, and the steps to implementing a graph in our code. To install this package, type "pip install networkx[default]" into the miniconda prompt. If you run into any problems, read the documentation that follows: https://networkx.org/documentation/stable/install.html. Matplot lib, requests, widgets, random, and iPython should already be installed as they were covered in class. 

In [14]:
import networkx as nx
from matplotlib import pyplot as plt
import requests
import ipywidgets as widgets
import random
from IPython.display import display, HTML
get_ipython().ast_node_interactivity = 'all'

top50Airports = ["ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "YYZ", "SEA", "CLT", "MCO", "MIA", "PHX", "EWR", "IAH", "MSP", "BOS", "DTW", "FLL", "ORL", "LGA", "PHL", "BWI", "SLC", "YVR", "DCA", "IAD", "MDW", "SAN", "HNL", "TPA", "PDX", "YUL", "YYC", "DAL", "STL", "BNA", "AUS", "HOU", "OAK", "SJC", "MSY", "RDU", "MCI", "SMF", "SNA", "CLE", "SAT", "PIT"]

dashboard = widgets.Output(layout={'border': '1.5px solid black'})
airline_code = widgets.Text(
    value='',
    description='Airline Code:',
    placeholder='Enter airline IATA code'
)

num_flights = widgets.IntSlider(
    value=0,
    min=1,
    max=50,
    step=1,
    description='# of Flights:',
    orientation='horizontal'
)

flight_options = widgets.Dropdown(
    options=sorted(top50Airports),
    description='DEP Airport:'
)

output = widgets.Output()

def getData(airIATA, depIATA):
    site = "https://app.goflightlabs.com/advanced-real-time-flights"
    access_key = " "
    parameters = {'access_key': access_key, 'airline_iata': airIATA, 'depIata' : depIATA}
    r = requests.get(site, parameters)
    flightData = r.json()
    arrivals = set()
    for i in range(len(flightData['data'])):
        if flightData['data'][i]['arrival']['iataCode'] in top50Airports and flightData['data'][i]['arrival']['iataCode'] not in arrivals:
            arrivals.add(flightData['data'][i]['arrival']['iataCode'])
    return list(arrivals)


def printGraph(depIATA, airIATA, numFlights):
    data = getData(airIATA, depIATA)
    random.shuffle(data)
    G = nx.DiGraph()
    G.add_node(depIATA)
    numFlights = min(numFlights, len(data))
    for i in range(numFlights):
        G.add_edge(depIATA, data[i])
    plt.figure(figsize=(8,8))
    plt.title(str(numFlights) + ' one-way flights from ' + depIATA)
    pos = nx.circular_layout(G)
    nx.draw(G, pos, node_size=800, with_labels=True)
    plt.show()
    display(HTML('<h3>Destinations:</h3>'))
    if numFlights == len(data):
        print(f"This graph contains all of the one-way flights from {depIATA} on {airIATA}.")
    else:
        print(f"Not happy with your destinations? Here is a list of all availible one-way flights from {depIATA} on {airIATA}")
        for i, element in enumerate(data):
            print(f"{i+1}. {element}")  

def on_button_clicked(_):
    with output:
        output.clear_output()
        printGraph(flight_options.value, airline_code.value, num_flights.value)


button = widgets.Button(
    description='Show Flights',
    layout=widgets.Layout(width='100%', height='100%')
)
button.on_click(on_button_clicked)

with dashboard: 
    display(airline_code, flight_options, num_flights, button, output)
dashboard

Output(layout=Layout(border_bottom='1.5px solid black', border_left='1.5px solid black', border_right='1.5px s…